In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import statsmodels.formula.api as smf

In [2]:
data = pd.read_csv('exercise3_data.csv', parse_dates = True, index_col = 0)

### I have given you a set of data that has *already* estimated the output gap and inflation differential between Europe and the US. I have also given you the change in the exchange rate associated with those observations under s_change. 

### Your goal will be to use this data and OLS to produce out of sample forecasts of the change in exchange rates. You will Then compute the MSE of youre forecasts and compare them to the Random Walk.

In [8]:
data.tail()

,gap_diff,inf_diff,s_change,forecast
2023-11-16,1.564830,-0.620009,-0.027016,-0.000281
2023-11-17,0.643254,-0.620009,-0.004896,-0.000697
2023-11-25,0.636683,0.357036,-0.004701,0.000598
2023-12-16,1.330083,0.357036,0.003162,0.002482
2023-12-26,1.320142,0.748037,-0.010851,0.003268


### 1.  Below I have given you a starting block of code for training your model on a rolling window and making an out of sample forecast one period ahead. 

### Complete the code below to generate a series of one-period ahead out-of-sample forecasts. You will Need to choose the right variable names to run the followig regression:
$$s\_change = \beta_1 + \beta_2 \text{Output Gap}_t + \beta_3 \text{Inflation Gap}_t + e_t$$

### Then set the appropriate training and forecasting window (for just the next observation)

In [3]:
data['forecast'] = 0.0
for i in range(120, len(data)):
    forecaster = smf.ols(formula = 's_change ~ inf_diff + gap_diff', data=data[i-120:i]).fit() #here we are using the data from the previous 120 days to predict the next day
    data.loc[data.index[i], 'forecast'] = forecaster.predict(data.iloc[i:i+1]).values #here we are predicting the next day. We are using the iloc function to get the data from the next day
    
#All in all, we are doing a rolling regression, where we are using the data from the previous 120 days to predict the next day. We are then using the iloc function to get the data from the next day and predict it. We are then storing the predicted value in the forecast column.

In [10]:
data[110:130]

,gap_diff,inf_diff,s_change,forecast
2007-01-16,-2.829613,0.081910,0.014904,0.000000
2007-01-17,-1.784869,0.081910,-0.001539,0.000000
2007-01-25,-1.766147,0.596858,0.000310,0.000000
2007-02-15,-3.347564,0.596858,-0.015119,0.000000
2007-02-17,-3.651477,0.596858,-0.001590,0.000000
2007-02-25,-3.597591,0.210444,-0.003188,0.000000
2007-03-18,-3.044587,0.210444,-0.008761,0.000000
2007-03-20,-2.039654,0.210444,-0.000146,0.000000
2007-03-28,-2.014959,0.548478,0.000000,0.000000
2007-04-17,-1.817355,0.548478,-0.019195,0.000000


In [7]:
# We are subsetting the datasset to the part that just conrtains our prediction
test_data = data.iloc[120:].copy()

### Choose how the signals are generated. We should go long if we forecast the exchange rate will increase and short if we forecast it will decrease. 

In [34]:
# Select the correct signal values
test_data['signal'] = np.where(test_data['forecast'] >= ___, ____, ____)

# note that I have shifted the return (-1) before hand, so we don't need to .shift() the signals
# The returns were shifted to be the appropriate label for the observations when we train our regression
test_data['strategy_return'] = (np.exp((test_data['signal']*test_data['s_change']).cumsum())-1)*100

In [ ]:
test_data['strategy_return'].plot(figsize = (10,5))

## Below is some code that computes the MSE of your exhcange rate forecasts

In [44]:
((test_data['s_change']-test_data['forecast'])**2).mean()

0.00029290371083852136

## Compute the mean squared error of the random walk forecast:

$$\text{MSE}_{rw} = \frac{1}{N} \sum_{t=1}^{N} s\_\text{change}_{t} - s\_\text{change}_{t-1})^2$$

Hint: Any easy way to do this is to apply a .shift()

In [45]:
# Compute the MSE of the RW forecast